In [25]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [26]:
PATH = "C:/input/Kaggle/DogsvsCats/"
sz=224
batch_size=64

In [51]:
import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout,Flatten,Dense,Dense,GlobalAveragePooling2D
from keras.models import Model,Sequential
from keras import backend as K
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

In [52]:
train_data_dir = '{}train/'.format(PATH)
validation_data_dir = '{}/valid'.format(PATH)
test_data_dir = '{}/test1'.format(PATH)

In [53]:
K.image_data_format() # 'channels_last'
preprocess_input # Preprocesses a tensor encoding a batch of images.

<function keras.applications.imagenet_utils.preprocess_input>

In [54]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_data_dir,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
    shuffle=False,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='binary')

Found 25000 images belonging to 2 classes.
Found 440 images belonging to 2 classes.


In [55]:
import cv2
im = cv2.imread(train_data_dir + 'dog.12485.jpg')
#print(im)

In [56]:
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [57]:
print(validation_generator.n)

440


In [58]:
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers: layer.trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [59]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=3, workers=4,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/3


Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\ac32\AppData\Local\Continuum\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Users\ac32\AppData\Local\Continuum\Anaconda3\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ac32\AppData\Local\Continuum\Anaconda3\lib\site-packages\keras\engine\training.py", line 612, in data_generator_task
    generator_output = next(self._generator)
  File "C:\Users\ac32\AppData\Local\Continuum\Anaconda3\lib\site-packages\keras\preprocessing\image.py", line 727, in __next__
    return self.next(*args, **kwargs)
  File "C:\Users\ac32\AppData\Local\Continuum\Anaconda3\lib\site-packages\keras\preprocessing\image.py", line 963, in next
    x = self.image_data_generator.standardize(x)
  File "C:\Users\ac32\AppData\Local\Continuum\Anaconda3\lib\site-packages\keras\preprocessing\image.py", line 486, in standardize
    x = self.preprocessing_func

ValueError: output of generator should be a tuple `(x, y, sample_weight)` or `(x, y)`. Found: None

In [ ]:
split_at = 140
for layer in model.layers[:split_at]: layer.trainable = False
for layer in model.layers[split_at:]: layer.trainable = True
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])    

In [ ]:
%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=1, workers=3,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(im)

In [ ]:
def transform_matrix_offset_center(matrix, x, y):
    o_x = float(x) / 2 + 0.5
    o_y = float(y) / 2 + 0.5
    offset_matrix = np.array([[1, 0, o_x], [0, 1, o_y], [0, 0, 1]])
    reset_matrix = np.array([[1, 0, -o_x], [0, 1, -o_y], [0, 0, 1]])
    transform_matrix = np.dot(np.dot(offset_matrix, matrix), reset_matrix)
    return transform_matrix

In [ ]:
from scipy import linalg
import scipy.ndimage as ndi
def apply_transform(x,
                    transform_matrix,
                    channel_axis=0,
                    fill_mode='nearest',
                    cval=0.):
    """Apply the image transformation specified by a matrix.

    # Arguments
        x: 2D numpy array, single image.
        transform_matrix: Numpy array specifying the geometric transformation.
        channel_axis: Index of axis for channels in the input tensor.
        fill_mode: Points outside the boundaries of the input
            are filled according to the given mode
            (one of `{'constant', 'nearest', 'reflect', 'wrap'}`).
        cval: Value used for points outside the boundaries
            of the input if `mode='constant'`.

    # Returns
        The transformed version of the input.
    """
    x = np.rollaxis(x, channel_axis, 0)
    final_affine_matrix = transform_matrix[:2, :2]
    final_offset = transform_matrix[:2, 2]
    channel_images = [ndi.interpolation.affine_transform(
        x_channel,
        final_affine_matrix,
        final_offset,
        order=0,
        mode=fill_mode,
        cval=cval) for x_channel in x]
    x = np.stack(channel_images, axis=0)
    x = np.rollaxis(x, 0, channel_axis + 1)
    return x

In [ ]:
x=im
zx=5;zy=1
zoom_matrix = np.array([[zx, 0, 0],
                            [0, zy, 0],
                            [0, 0, 1]])
h,w =im.shape[0],im.shape[1] #(374, 500, 3)
transform_matrix = transform_matrix_offset_center(zoom_matrix, h, w)
x = apply_transform(x, transform_matrix, channel_axis=2)
#print(x)
x.shape,plt.imshow(x)

In [ ]:
K.image_data_format()

In [50]:
import os
testpath = 'E:/tmp/test/'
aa =sorted(os.walk(testpath, followlinks=True), key=lambda tpl: tpl[0])
type(aa);aa

[('E:/tmp/test/', ['temp', 'temp2'], ['2018_10_09_17_43_17.pdf', 'file.txt']),
 ('E:/tmp/test/temp', [], ['New Text Document.txt']),
 ('E:/tmp/test/temp2', [], ['New Text Document2.txt'])]

[]

In [ ]:
print(train_data_dir + 'dog.12485.jpg')